In [ ]:
import pickle
with open('ingr_map.pkl', 'rb') as f:
    data = pickle.load(f)
mapping_dups = data[["id", "replaced"]].copy()
mapping_unordered = mapping_dups.drop_duplicates(subset='id')
mapping = mapping_unordered.sort_values(by='id')

In [1]:
%pip install pandas

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

raw_recipes = pd.read_csv('static/data/RAW_recipes_cut.csv')
pp_recipes = pd.read_csv('static/data/PP_recipes_cut.csv')
interactions = pd.read_csv('static/data/RAW_interactions_cut.csv')

In [16]:
import ast
import re

In [30]:
def remove_special_characters(text):
    # Define a regular expression pattern to match special characters except single quotes
    pattern = r"[^a-zA-Z0-9\s'.,!?]"
    # Replace special characters with an empty string
    cleaned_text = re.sub(pattern, '', text)
    cleaned_text = cleaned_text.replace('\n', '')
    return cleaned_text

In [31]:
merged_df = pd.merge(raw_recipes, pp_recipes, on='id', how='inner')
recipes = merged_df[['id', 'name', 'minutes', 'ingredient_ids', 'ingredients', 'steps','description']]
recipes_review_merge = pd.merge(recipes, interactions, left_on='id',right_on='recipe_id', how='inner')
recipes_review = recipes_review_merge[['id', 'name', 'minutes', 'ingredients', 'steps','description', 'rating','review']]
final = recipes_review.sort_values(by='id') #.drop_duplicates(subset='id')
final['avg_rating']  = final.groupby(['id'])['rating'].transform('mean')
final = final.groupby(['id', 'name', 'minutes', 'ingredients', 'steps', 'description', 'avg_rating'])['review'].agg(lambda x: 'REV'.join(str(review) for review in x))
final = final.reset_index()
final = final.rename(columns={'review': 'reviews'})
final['ingredients'] = final['ingredients'].apply(lambda x: ', '.join(ast.literal_eval(x))).str.replace("'", "''")
final['steps'] = final['steps'].apply(lambda x: ', '.join(ast.literal_eval(x))).str.replace("'", "''")
final['name'] = final['name'].str.replace("'", "''")
final['description'] = final['description'].str.replace("'", "''")
final['reviews'] = final['reviews'].str.replace("'", "''")

final['steps'] = final['steps'].apply(remove_special_characters)
final['decsription'] = final['description'].apply(remove_special_characters)
final['reviews'] = final['reviews'].apply(remove_special_characters)
last_column_index = final.shape[1] - 1
final = final.drop(final.columns[last_column_index], axis=1)

final

,id,name,minutes,ingredients,steps,description,avg_rating,reviews
0,38,low fat berry blue frozen dessert,1485,"blueberries, granulated sugar, vanilla yogurt,...","toss 2 cups berries with sugar, let stand for ...","this is yummy and low-fat, it always turns out...",4.250000,"Yummy, yummy, yummy! I am a big fan of fruit a..."
1,40,best lemonade,35,"sugar, lemons, rind of, fresh water, fresh lem...","into a 1 quart jar with tight fitting lid , pu...",this is from one of my first good house keepi...,4.333333,This was awesome!! it was REALLY refreashing!R...
2,45,buttermilk pie with gingersnap crumb crust,80,"sugar, margarine, egg, egg whites, flour, salt...","preheat oven to 350f, make pie crust , using 8...",yum,2.666667,This was my first buttermilk pie I had never h...
3,49,chicken breasts lombardi,75,"fresh mushrooms, butter, boneless skinless chi...",cook mushrooms in 2 tbsp butter in a large ski...,"cheese, chicken, and mushrooms with marsala.",4.350000,DH and I thought this was delicious. I think ...
4,58,low fat burgundy beef vegetable stew,164,"beef eye round, vegetable oil, dried thyme lea...","trim fat from beef , cut into 1inch pieces, in...",(high fiber),4.428571,A very good beef stew recipe! I used a regular...
...,...,...,...,...,...,...,...,...
431,3185,chicken mole ii,0,"unsifted flour, chili powder, salt, pepper, on...","chicken in a large bowl or plastic bag , combi...",this mole is milder in flavor than a tradition...,5.000000,"Yum, rich and tasty and not too spicy. Great ..."
432,3186,herbed gougere puffs,75,"water, butter, salt, all-purpose flour, eggs, ...","in 2quart saucepan , heat 1 cup water , butter...",this is an adopted recipe that i''ve not yet h...,4.500000,This recipe is perfect for entertaining. I cr...
433,3189,walnut brownie cheesecake,95,"fine chocolate wafer crumbs, margarine, light ...",lightly grease sides of 8 or 9inch springform ...,this recipe is one of my adoptees. after makin...,5.000000,Outstanding recipe! I loved everything about t...
434,3200,cranberry coconut muffins,30,"all-purpose flour, granulated sugar, sweetened...","in bowl , stir together flour , sugar , 1 2 c...",yum,4.500000,I more than halved the sugar to 13 cup and fin...


In [20]:
final.loc[0, 'reviews']

'very good!! the directions were alittle weird though...REVThis does taste great! I think the recipe needs a bit less berries and a little bit more yogurt though. The result is still very berry!REVTasty and refreshing! I love the creamy flavor the vanilla yogurt adds. I didn039t put mine in a mold, but instead just served it up like ice cream.'

In [34]:
with open('../init.sql', 'w') as f:
    # Write the DROP TABLE statement
    f.write("DROP TABLE IF EXISTS recipes;\n\n")
    
    # Write the CREATE TABLE statement
    f.write("CREATE TABLE recipes(\n")
    f.write("    id int,\n")
    f.write("    nam TEXT,\n")
    f.write("    minutes int,\n")
    f.write("    ingredients TEXT,\n")
    f.write("    steps TEXT,\n")
    f.write("    descr TEXT,\n")
    f.write("    avgrating float,\n")
    f.write("    reviews TEXT\n")
    f.write(");\n\n")
    
    # Iterate over DataFrame rows and write INSERT INTO statements
    for idx, row in final.iterrows():
        f.write(f"INSERT INTO recipes VALUES ({row['id']}, '{row['name']}', {row['minutes']}, '{row['ingredients']}', '{row['steps']}', '{row['description']}', {row['avg_rating']}, '{row['reviews']}');\n")